In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password

# Diversity Dataset

First, we loaded our most complex dataset: the diversity index CSV.

To transform the data, we first split the location column into separate county and state columns to allow interaction with the other two datasets.

State averages were also included in the dataset and listed as a full state name in the location column. Since we only wanted data by county, once locations were split into county and state, we dropped all rows that had state only info.
    For example, most locations followed the “county, state abbreviation” format (Alameda County, CA), however state averages were listed by just the state name in all caps (TEXAS). Once counties and states were split into separate columns, those rows with only a state average resulted in a value of ‘NaN’ in column our new State column. We were then able to drop all the state average rows by dropping all rows with a State value of ‘NaN’.

We verified this was successful by checking that no results were found when searching our new County column by a full state name, in this case ‘TEXAS’.

In [ ]:
diversity_csv = "Resources/diversityindex.csv"
diversity_df = pd.read_csv(diversity_csv)
diversity_df

In [ ]:
# Create a a new df with info we need 

# Split the County coloum into State and County
new_df = diversity_df
new_df['County'], new_df['State'] = diversity_df['Location'].str.split(',').str
new_df.tail()

# Drops all rows that have NaN as County as States got split into County above. 
new_df = new_df.dropna()

#checking if this worked and drop was sucessful in removing STATe names from county coloum 

# dsd = df[df['County'] == 'TEXAS']
# dsd.tail()

new_df.head()

In [ ]:
# drop uneccessary column "Location" - replaced by county, state columns
diversity_df = new_df.drop(['Location'], axis=1)

# rename columns for compatibility with SQL
diversity_df = diversity_df.rename(columns={'Diversity-Index': 'diversity_index',
                                            'Black or African American alone, percent, 2013': 'black',
                                           'American Indian and Alaska Native alone, percent, 2013': 'native_am',
                                           'Asian alone, percent, 2013': 'asian',
                                           'Native Hawaiian and Other Pacific Islander alone, percent,': 'hawaiian',
                                           'Two or More Races, percent, 2013': 'mixed',
                                           'Hispanic or Latino, percent, 2013': 'latinx',
                                           'White alone, not Hispanic or Latino, percent, 2013': 'white',
                                           'County': 'county',
                                           'State': 'state'})

diversity_df

# Unemployment Dataset

We then loaded the unemployment dataset. We discovered that this dataset was organized quite differently from the others in that it was broken down first by year, then by month, then by state and county. This meant we had unemployment data broken out by month from 1990 up to 2016, resulting in a far larger dataset than our other two CSVs. To deal with this discrepancy, we averaged the unemployment rate by grouping across county and state.

For this dataset, states were also not in a format which was compatible with our other two datasets. To fix this, we located “chocolate cake recipe” code for quickly converting full state names into state abbreviations and applied this to our dataset.

In [ ]:
unemp_csv = "Resources/unemployment.csv"
unemp_df = pd.read_csv(unemp_csv)

unemp_df.head()

In [ ]:
unemp_df = unemp_df[['County', 'State', 'Rate']]
unemp_df.head(10)

In [ ]:
us_state_abbrev = {
    
'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'}


In [ ]:
unemp_df['State'] = unemp_df['State'].map(us_state_abbrev).fillna(unemp_df['State'])

# rename columns for compatibility with SQL
unemp_df = unemp_df.rename(columns={'County': 'county', 'State': 'state', 'Rate': 'rate'})

# drop duplicate rows
unemp_df = unemp_df.drop_duplicates(subset=['county', 'state'])

unemp_df.head(10)

# Median Income Dataset

Finally, we loaded in our median income dataset. This was our cleanest dataset and required very little transformation on our part to be compatible with the other two. County and state abbreviations were already in place and broken out into separate columns. Median household income was also already in a usable format. The only necessary cleanup was to drop columns that contained either redundant info (County-State and State (full name) or info not needed (Population).

In [ ]:
median_csv = "Resources/medianincome.csv"
median_df = pd.read_csv(median_csv)
median_df.info()

In [ ]:
median_df.info()

In [ ]:
median_df = median_df[['County',"State Code", 'Population',"Median household income"]]

# rename columns for compatibility with SQL
median_df = median_df.rename(columns={'County': 'county', 'State Code': 'state', 'Population': 'population', 'Median household income': 'median_household_income'})

median_df.head()

# Load to SQL

Once all three datasets were extracted and transformed, we loaded them into three tables within an SQL database. For future hypothetical analysis, data across the three tables could be compared using the composite keys for each table of county and state.

In [ ]:
# Connect to database
rds_connection_string = f"{username}:{password}@localhost:5432/ETL_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [ ]:
# Check for tables
engine.table_names()

In [ ]:
# Use pandas to load converted dataframes into ETL_db
median_df.to_sql(name='income', con=engine, if_exists='append', index=False)

In [ ]:
diversity_df.to_sql(name='diversity', con=engine, if_exists='append', index=False)

In [ ]:
unemp_df.to_sql(name='unemployment', con=engine, if_exists='append', index=False)

# Confirm load

In [ ]:
pd.read_sql_query('select * from income', con=engine).head()

In [ ]:
pd.read_sql_query('select * from diversity', con=engine).head()

In [ ]:
pd.read_sql_query('select * from unemployment', con=engine).head()